In [1]:
import numpy as np
import pandas as pd
import pickle

#Packages needed for getting the properties we need
import mofid
from mofid.run_mofid import cif2mofid
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import Counter
import concurrent.futures

In [2]:
df = pd.read_csv('CoRE_or_QMOF.csv')
df

,CSD code,Largest included sphere diameter (A),Total volumetric surface area (m^2/m^3),CO2 uptake at LP (mol/kg),CO2 uptake at HP (mol/kg),CH4 uptake at HP (mol/kg),logKH_CO2,Band gap (eV)
0,QONLAI,6.10211,1909.4500,0.624173,10.315793,8.403936,-4.376034,0.485636
1,BUKYAJ,5.10209,752.8280,2.442599,6.769549,6.642334,-3.589223,3.866396
2,QAGDIO,12.81046,1833.5900,0.440013,16.889565,13.095273,-4.533132,0.340761
3,XUYDIF,3.88622,7.3351,0.363202,1.301727,1.374730,-4.342234,1.785168
4,YEZFIU,7.76134,1275.6500,0.318448,6.437679,5.797507,-4.685284,4.751611
...,...,...,...,...,...,...,...,...
24206,QAZKIN,4.20402,NaN,NaN,NaN,NaN,NaN,3.292749
24207,WEZXAB01,2.28741,NaN,NaN,NaN,NaN,NaN,4.832323
24208,XANKED,1.94585,NaN,NaN,NaN,NaN,NaN,0.418669
24209,AKINIS,2.42584,NaN,NaN,NaN,NaN,NaN,2.156904


In [3]:
with open('CSD code map/CSD_to_CORE.pickle', 'rb') as handle:
    CSD_to_CORE = pickle.load(handle)

with open('CSD code map/CSD_to_QMOF.pickle', 'rb') as handle:
    CSD_to_QMOF = pickle.load(handle)

In [4]:
with open('CSD code map/sets/CoRE_QMOF_intersection.pickle', 'rb') as handle:
    CORE_QMOF_int = pickle.load(handle)

with open('CSD code map/sets/CoRE_only.pickle', 'rb') as handle:
    CORE_only = pickle.load(handle)

with open('CSD code map/sets/QMOF_only.pickle', 'rb') as handle:
    QMOF_only = pickle.load(handle)

In [11]:
len(QMOF_only), len(CORE_only), len(CORE_QMOF_int)

(14885, 8162, 1164)

In [6]:
#make map of: [Metal atom type, SMILES of linker, MOFid, space group, molecular formula, crystal system]
int_props = {} #{ref code : [map]}
core_props = {} #core only
qmof_props = {} #qmof only

In [26]:
def getMOFid(directory):
    mofid_dict = cif2mofid(directory)
    return mofid_dict['mofid'], mofid_dict['smiles_nodes'], mofid_dict['smiles_linkers'], mofid_dict['smiles']

def spaceGroup(structure):
    analyzer = SpacegroupAnalyzer(structure)
    return analyzer.get_space_group_symbol()

def crystalSystem(structure):
    analyzer = SpacegroupAnalyzer(structure)
    return analyzer.get_crystal_system()

def molecularFormula(structure):
    formula = structure.composition.alphabetical_formula.replace(" ", "")
    return formula

In [29]:
def process_code(code):
    try:
        actualCode = CSD_to_CORE[code]
        directory = f'CoRE-MOF Data/cifs/{actualCode}.cif'
        MOFid, smiles_nodes, smiles_linkers, smiles = getMOFid(directory=directory)
        structure = Structure.from_file(directory)
        space_group = spaceGroup(structure=structure)
        crys_sys = crystalSystem(structure=structure)
        formula = molecularFormula(structure=structure)
        return code, [MOFid, smiles_nodes, smiles_linkers, smiles, formula, space_group, crys_sys]
    except:
        return code, np.nan
    
def process_fold(fold):
    fold_results = {}
    for code in fold:
        code, result = process_code(code)
        fold_results[code] = result
    return fold_results

def split_into_folds(data, n_folds):
    return np.array_split(data, n_folds)

In [15]:
int_props = {}
n_folds = 10

folds = split_into_folds(CORE_QMOF_int, n_folds)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_fold = {executor.submit(process_fold, fold): fold for fold in folds}
    for future in concurrent.futures.as_completed(future_to_fold):
        fold_results = future.result()
        int_props.update(fold_results)

In [20]:
len(int_props)

1164

In [23]:
with open('CSD code map/property maps/int_props.pickle', 'wb') as handle:
    pickle.dump(int_props, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
core_props = {}
n_folds = 10

folds = split_into_folds(CORE_only, n_folds)

In [31]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_fold = {executor.submit(process_fold, fold): fold for fold in folds}
    for future in concurrent.futures.as_completed(future_to_fold):
        fold_results = future.result()
        core_props.update(fold_results)

*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1)


1
1


*** Open Babel Error  in WriteSystre
  Found two neighboring 2-c sites.  Flagging the cgd output to get an error instead of the incorrect topology.
Note: Rerun ExportSystre() with simplify_two_conn=false if an unsimplified net is useful.
*** Open Babel Error  in WriteSystre
  Found two neighboring 2-c sites.  Flagging the cgd output to get an error instead of the incorrect topology.
Note: Rerun ExportSystre() with simplify_two_conn=false if an unsimplified net is useful.
*** Open Babel Error  in WriteSystre
  Found two neighboring 2-c sites.  Flagging the cgd output to get an error instead of the incorrect topology.
Note: Rerun ExportSystre() with simplify_two_conn=false if an unsimplified net is useful.
*** Open Babel Error  in WriteSystre
  Found two neighboring 2-c sites.  Flagging the cgd output to get an error instead of the incorrect topology.
Note: Rerun ExportSystre() with simplify_two_conn=false if an unsimplified net is useful.


1
1
1


*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): C(3)


...

1
1
1


*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): N(2)


1


/home/sartaaj/.local/lib/python3.10/site-packages/pymatgen/io/cif.py:1200: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
*** Open Babel Warning  in InChI code
  #0 :Accepted unusual valence(s): O(1)


1


In [32]:
print(len(core_props))

with open('CSD code map/property maps/core_props.pickle', 'wb') as handle:
    pickle.dump(core_props, handle, protocol=pickle.HIGHEST_PROTOCOL)

8162


In [15]:
qmof_df = pd.read_csv('QMOF Data/qmof.csv')
qmof_df

/tmp/ipykernel_1970329/1168522333.py:1: DtypeWarning: Columns (40,41,43,44,45,46,47,49,50,51,52,53,55,56,57,65,66,67,68,69,77,78,79,80,81,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  qmof_df = pd.read_csv('QMOF Data/qmof.csv')


,qmof_id,name,info.formula,info.formula_reduced,info.mofid.mofid,info.mofid.mofkey,info.mofid.smiles_nodes,info.mofid.smiles_linkers,info.mofid.smiles,info.mofid.topology,...,outputs.hse06.energy_elec,outputs.hse06.net_magmom,outputs.hse06.bandgap,outputs.hse06.cbm,outputs.hse06.vbm,outputs.hse06.directgap,outputs.hse06.bandgap_spins,outputs.hse06.cbm_spins,outputs.hse06.vbm_spins,outputs.hse06.directgap_spins
0,qmof-8a95c27,ABACUF01_FSR,Ba2CuC6H14O16,Ba2CuC6H14O16,NaN,NaN,"['O', '[Ba]', '[Cu]']",['[O-]C=O'],O.[Ba].[Cu].[O-]C=O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,qmof-019ba28,ABALOF_FSR,Cu12C36H56I16N4S4,Cu3C9H14I4NS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,qmof-830ed1c,ABAVIJ_FSR,Co4C48H32N8O16,CoC12H8N2O4,[Co].[O-]C(=O)c1ccncc1 MOFid-v1.rtl.cat0,Co.TWBYWOBDOCUKOW.MOFkey-v1.rtl,['[Co]'],['[O-]C(=O)c1ccncc1'],[Co].[O-]C(=O)c1ccncc1,rtl,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,qmof-5bd4a24,ABAVOP_FSR,Co4C48H32N8O16,CoC12H8N2O4,[Co].[O-]C(=O)c1ccncc1 MOFid-v1.rtl.cat0,Co.TWBYWOBDOCUKOW.MOFkey-v1.rtl,['[Co]'],['[O-]C(=O)c1ccncc1'],[Co].[O-]C(=O)c1ccncc1,rtl,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,qmof-644aab4,ABAXUZ_FSR,Zn2C50H32N6O8S4,ZnC25H16N3O4S2,NaN,NaN,['[Zn][Zn]'],"['[O-]C(=O)c1cccc(c1)c1nccs1', 'n1ccc(cc1)c1cc...",[O-]C(=O)c1cccc(c1)c1nccs1.[Zn][Zn].n1ccc(cc1)...,NaN,...,-811.553858,0.0,2.901747,2.246703,-0.655044,True,"[None, None]","[None, None]","[None, None]","[None, None]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20370,qmof-7aebbbb,tobacco_srsb_sym_3_on_2_sym_3_mc_0_L_2,Cu12C84H60N24,CuC7H5N2,N1=C[C](C=N1)C=Cc1cc(C=CC2=C[N]N=C2)cc(c1)C=CC...,Cu.IBPUNEAULYEGJU.MOFkey-v1.srs,['[Cu]'],['N1=C[C](C=N1)C=Cc1cc(C=CC2=C[N]N=C2)cc(c1)C=...,N1=C[C](C=N1)C=Cc1cc(C=CC2=C[N]N=C2)cc(c1)C=CC...,srs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20371,qmof-9a04c15,tobacco_srsb_sym_3_on_2_sym_3_mc_0_L_6,Cu12C84H48N60,CuC7H4N5,N1=C[C](C=N1)n1nnc(c1)c1cc(cc(c1)c1nnn(c1)C1=C...,Cu.JWLDCPHWRGZUAB.MOFkey-v1.srs,['[Cu]'],['N1=C[C](C=N1)n1nnc(c1)c1cc(cc(c1)c1nnn(c1)C1...,N1=C[C](C=N1)n1nnc(c1)c1cc(cc(c1)c1nnn(c1)C1=C...,srs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20372,qmof-0dce90f,tobacco_srsb_sym_3_on_2_sym_3_mc_0__,Cu12C60H36N24,CuC5H3N2,N1=C[C](C=N1)c1cc(cc(c1)C1=CN=N[CH]1)C1=C[N]N=...,Cu.PJSMFZDMZONQKK.MOFkey-v1.srs,"['[Cu]', '[Cu][Cu]']",['N1=C[C](C=N1)c1cc(cc(c1)C1=CN=N[CH]1)C1=C[N]...,N1=C[C](C=N1)c1cc(cc(c1)C1=CN=N[CH]1)C1=C[N]N=...,srs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20373,qmof-955fe88,tobacco_srsb_sym_3_on_4_sym_3_mc_0_L_2,Cu12C112H72N24,Cu3C28H18N6,N1=C[C](C=N1)C=CC1=CC2=CC(=CC3=CC(=CC(=C1)[C]2...,Cu.WCJPEIPZJUESBA.MOFkey-v1.srs,['[Cu]'],['N1=C[C](C=N1)C=CC1=CC2=CC(=CC3=CC(=CC(=C1)[C...,N1=C[C](C=N1)C=CC1=CC2=CC(=CC3=CC(=CC(=C1)[C]2...,srs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
CSD_to_QMOF_id = {}

for i in QMOF_only:
    refcode = CSD_to_QMOF[i]
    qmof_id = qmof_df[qmof_df['name'] == refcode]['qmof_id'].values[0]
    CSD_to_QMOF_id[i] = qmof_id

In [28]:
def process_code_QMOF(code):
    try:
        actualCode = CSD_to_QMOF_id[code]
        directory = f'QMOF Data/relaxed_structures/{actualCode}.cif'
        MOFid, smiles_nodes, smiles_linkers, smiles = getMOFid(directory=directory)
        structure = Structure.from_file(directory)
        space_group = spaceGroup(structure=structure)
        crys_sys = crystalSystem(structure=structure)
        formula = molecularFormula(structure=structure)
        return code, [MOFid, smiles_nodes, smiles_linkers, smiles, formula, space_group, crys_sys]
    except:
        return code, np.nan
    
def process_fold_QMOF(fold):
    fold_results = {}
    for code in fold:
        code, result = process_code_QMOF(code)
        fold_results[code] = result
    return fold_results

def split_into_folds(data, n_folds):
    return np.array_split(data, n_folds)

In [29]:
qmof_props = {}
n_folds = 10

folds = split_into_folds(QMOF_only, n_folds)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_fold = {executor.submit(process_fold_QMOF, fold): fold for fold in folds}
    for future in concurrent.futures.as_completed(future_to_fold):
        fold_results = future.result()
        qmof_props.update(fold_results)

In [31]:
print(len(qmof_props))

with open('CSD code map/property maps/qmof_props.pickle', 'wb') as handle:
    pickle.dump(qmof_props, handle, protocol=pickle.HIGHEST_PROTOCOL)

14885


In [32]:
qmof_props['ABACUF01']

['O.[Ba].[Cu].[O-]C=O MOFid-v1.UNKNOWN.cat0;qmof-8a95c27',
 ['O', '[Ba]', '[Cu]'],
 ['[O-]C=O'],
 'O.[Ba].[Cu].[O-]C=O',
 'Ba2C6Cu1H14O16',
 'P-1',
 'triclinic']